In [1]:
from typing import List
from spacy.tokens import Doc, Span
from utils import load_models, print_clusters, print_comparison
import pandas as pd
from openie import StanfordOpenIE

In [31]:
predictor, nlp = load_models()

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## The problem
AllenNLP coreference resolution models seems to find better clusters - numerous clusters that are usually more accurate than the ones found by Huggingface NeuralCoref model. However, the biggest problem lies in the next step - the step of replacing found mentions with the most meaningfull spans from each clusters (that we call the "heads"). We've found a couple of easy-to-fix problems which seem to lead to errors most often. Our ideas can be summed up as:
- not resolving coreferences in the clusters that doesn't contain any noun phrases (usually it comes down to the clusters composed only of pronouns),
- chosing the head of the cluster which is a noun phrase (isn't a pronoun),
- resolving only the inner span in the case of nested coreferent mentions.

We show all of our improvements by example - for more details please refer to our [third and last blog post](). If you're interested in problems themselves our [second blog post](https://neurosys.com/article/most-popular-frameworks-for-coreference-resolution/) contains all the definitions and theory.

## Original AllenNLP impelemntation of the `replace_corefs` method

We extract the main "logic" into the separate function that will be used in our every method as we leave the core of AllenNLP's logic untouched. So as for now, we will compare our solutions to the `original_replace_corefs` method implemented in AllenNLP `coref.py` (we've just copied it here explicitly in order to compare with the improved method we propose).

In [32]:
def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved


def original_replace_corefs(document: Doc, clusters: List[List[List[int]]]) -> str:
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
        mention_span = document[mention_start:mention_end]

        for coref in cluster[1:]:
            core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

## Improvements
### Redundant clusters - lack of a meaningfull mention that could become the head
We completely ignore (we don't resove them at all) the clusters that doesn't contain any noun phrase.

In [33]:
def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:  # if there is at least one noun phrase...
            mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
            mention_span = document[mention_start:mention_end]

            for coref in cluster[1:]:
                core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

**Example**  
We want to take our code and create a game. Let's remind ourselves how to do that.  

**Original coreference resolution clusters** (by AllenNLP)   
We --> We; our; 's; ourselves  
create --> create; that

In [5]:
text = "We want to take our code and create a game. Let's remind ourselves how to do that."
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [6]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

~~~ AllenNLP original replace_corefs ~~~
We want to take We's code and create a game. LetWe remind We how to do create.

~~~ Our improved replace_corefs ~~~
We want to take our code and create a game. Let's remind ourselves how to do that.


### Cataphora problem - choosing the wrong cluster *head*
We redefine the span that becomes a cluster head. Instead of choosing the first mention in the cluster, we pick the one that is the first **noun phrase in the cluster** - we define it as the first span that contains a noun.

In [34]:
def get_cluster_head(doc: Doc, cluster: List[List[int]], noun_indices: List[int]):
    head_idx = noun_indices[0]
    head_start, head_end = cluster[head_idx]
    head_span = doc[head_start:head_end+1]
    return head_span, [head_start, head_end]


def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention:  # we don't replace the head itself
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

**Example**   
"He is a great actor!", he said about John Travolta.

**Original coreference resolution clusters** (by AllenNLP)  
He --> He; John Travolta

In [7]:
text = '"He is a great actor!", he said about John Travolta.'
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [9]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

~~~ AllenNLP original replace_corefs ~~~
"He is a great actor!", he said about He.

~~~ Our improved replace_corefs ~~~
"John Travolta is a great actor!", he said about John Travolta.


### Nested coreferent mentions
I the case of nested mentions we choose to resolve the inner span (e.g. for the mention "his dog" the token *his* can be the inner span). That just means we don't want to resolve outer spans.

In [35]:
def is_containing_other_spans(span: List[int], all_spans: List[List[int]]):
    return any([s[0] >= span[0] and s[1] <= span[1] and s != span for s in all_spans])


def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)
    all_spans = [span for cluster in clusters for span in cluster]  # flattened list of all spans

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_span, mention = get_cluster_head(document, cluster, noun_indices)

            for coref in cluster:
                if coref != mention and not is_containing_other_spans(coref, all_spans):
                    core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

**Example**  
Anna likes Tom. Tom is Anna's brother. Her brother is tall.

**Original coreference resolution clusters** (by AllenNLP)  
Tom --> Tom; Tom; Her brother  
Anna --> Anna; Anna 's; Her

In [9]:
text = "Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV."
clusters = predictor.predict(text)['clusters']
doc = nlp(text)

In [ ]:
# Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV.
# alan shepard was born in new hampshire and died in california. he graduated from nwc, m.a. in 1957, was a test pilot and crew member of the apollo 14, which was operated by nasa.

In [34]:
print('~~~ original text ~~~\n', text ,'\n')
# print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))
print('\n',improved_replace_corefs(doc, clusters))

~~~ original text ~~~
 Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV. 


 Fred Astaire was born in Omaha, Nebraska, to Joha

In [36]:
def resolve_ref(text):
    global count
    count += 1
    if type(text)==str and len(text) < 1200: 
        print('accepting ',count)
        clusters = predictor.predict(text)['clusters']
        doc = nlp(text) 
        return improved_replace_corefs(doc, clusters)
    else:
#         print('discarding ',count)
        return 

In [37]:
#read data in csv format
#make a list of coref-resolved text and append it to dataframe
count = 0
def read(df,column,output_file):
    th = 1000
    ll = 0
    ul = ll + th
    resolved_text_list = []
#     df = pd.read_csv('data/bio.csv')
    print(len(df))
#     display(df.head())
    while len(resolved_text_list)<th:
        trimmed_df = df[ll:ul]
        ll = ul
        ul = ul + th
        for text in trimmed_df[column]:
            resolved_text = resolve_ref(text)
            if resolved_text:
                resolved_text_list.append(resolved_text)
            if len(resolved_text_list) >= th:
                break
    resolved_df = pd.DataFrame(list(zip(resolved_text_list)),columns = ['resolved_text'])
    display(resolved_df.head())
    print(len(resolved_df))
#     resolved_df.to_csv('data/resolved_bio.csv',index=False)
    resolved_df.to_csv(output_file,index=False)

In [ ]:
df = pd.read_csv('data/wiki_summary.csv')
column = 'summary'
output_file = 'data/wiki_summary_resolved.csv'
read(df,column,output_file)

1000
accepting  1
accepting  2
accepting  3
accepting  4
accepting  5
accepting  6
accepting  7
accepting  8
accepting  9
accepting  10
accepting  11
accepting  12
accepting  13
accepting  14
accepting  15
accepting  16
accepting  17
accepting  18
accepting  19
accepting  20
accepting  21
accepting  22
accepting  23
accepting  24
accepting  25
accepting  26
accepting  27
accepting  28
accepting  29
accepting  30
accepting  31
accepting  32
accepting  33
accepting  34
accepting  35
accepting  36
accepting  37
accepting  38
accepting  39
accepting  40
accepting  41
accepting  42
accepting  43
accepting  44
accepting  45
accepting  46
accepting  47
accepting  48
accepting  49
accepting  50
accepting  51
accepting  52
accepting  53
accepting  54
accepting  55
accepting  56
accepting  57
accepting  58
accepting  59
accepting  60
accepting  61
accepting  63
accepting  64
accepting  65
accepting  66
accepting  67
accepting  68
accepting  69
accepting  70
accepting  71
accepting  72
accepting 

Exception ignored in: <function StanfordOpenIE.__del__ at 0x7f05e5bf0700>
Traceback (most recent call last):
  File "/home/anurag/.local/lib/python3.8/site-packages/openie/openie.py", line 102, in __del__
    del os.environ['CORENLP_HOME']
  File "/usr/lib/python3.8/os.py", line 691, in __delitem__
    raise KeyError(key) from None
KeyError: 'CORENLP_HOME'


accepting  93
accepting  94
accepting  95
accepting  96
accepting  97
accepting  98
accepting  101
accepting  102
accepting  103
accepting  104
accepting  105
accepting  106
accepting  107
accepting  108
accepting  109


Our last version of the `improved_replace_corefs` contains all of ours refinements. And that's it! You can now use it in your project with or without the intersection strategies (see the other Jupyter Notebook file). Good luck!

In [37]:
#standford openie
referenced_text = improved_replace_corefs(doc, clusters)
with StanfordOpenIE() as client:
    text = 'Barack Obama was born in Hawaii. Richard Manning wrote this sentence.'
    print('Text: %s.' % referenced_text)
    for triple in client.annotate(referenced_text):
        print('|-', triple)

#     graph_image = 'graph.png'
#     client.generate_graphviz_graph(text, graph_image)
#     print('Graph generated: %s.' % graph_image)

#     with open('corpus/pg6130.txt', 'r', encoding='utf8') as r:
#         corpus = r.read().replace('\n', ' ').replace('\r', '')

#     triples_corpus = client.annotate(corpus[0:50000])
#     print('Corpus: %s [...].' % corpus[0:80])
#     print('Found %s triples in the corpus.' % len(triples_corpus))
#     for triple in triples_corpus[:3]:
#         print('|-', triple)

Text: Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred Astaire entered show business at age 5. Fred Astaire was successful both in vaudeville and on Broadway in partnership with Fred Astaire's sister, Adele Astaire. After his sister, Adele Astaire retired to marry in 1932, Fred Astaire headed to Hollywood. Signed to RKO, Fred Astaire was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In RKO's Carioca (1933), Fred Astaire began Fred Astaire's highly successful partnership with Ginger Rogers, with whom Fred Astaire danced in 9 RKO pictures. During these years, Fred Astaire was also active in recording and radio. On film, Fred Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which Fred Astaire opened Fred Astaire Dance Studios, Fred Astaire returned to film to star in more musicals through 1957. Fred Astaire subse

In [1]:
s = "Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. Fred entered show business at age 5. He was successful both in vaudeville and on Broadway in partnership with his sister, Adele Astaire. After Adele retired to marry in 1932, Astaire headed to Hollywood. Signed to RKO, he was loaned to MGM to appear in La danza di Venere (1933) before starting work on RKO's Carioca (1933). In the latter film, he began his highly successful partnership with Ginger Rogers, with whom he danced in 9 RKO pictures. During these years, he was also active in recording and radio. On film, Astaire later appeared opposite a number of partners through various studios. After a temporary retirement in 1945-7, during which he opened Fred Astaire Dance Studios, Astaire returned to film to star in more musicals through 1957. He subsequently performed a number of straight dramatic roles in film and TV."
print(len(s))

923


In [36]:
#clean resolved bio
# d_df = pd.read_csv('data/resolved_bio.csv')
# print(len(d_df))
# d_df = d_df.dropna()
# print(len(d_df))
# d_df.to_csv('data/clean_resolved_bio.csv',index = False)
d_df = pd.read_csv('data/clean_resolved_bio.csv')
d_df.columns = ['unamed','resolved_text']
d_df = d_df.drop(columns = ['unamed'])

In [ ]:
print()

In [33]:
d_df.head()

,Unnamed: 0,bio
0,0,"Fred Astaire was born in Omaha, Nebraska, to J..."
1,9,One of Hollywood's preeminent male stars of al...
2,10,Born to Alice Cooper and Charles Cooper. Gary ...
3,15,"Georges Delerue was born on March 12, 1925 in ..."
4,23,"Born in London, England, John Gielgud trained ..."


In [2]:
def relation_camel_casing(words):
    s = "".join(word[0].upper() + word[1:].lower() for word in words)
    return s[0].lower() + s[1:]

In [21]:
# fun(list_of_dic):
# output: list_of_dic

# first make a set of all relations:
# pick one relation from the set and iterate over the list of dics
# 	if rel == picked rel
# 		merge the objects with comma
# 		add in new dictionary
def triplet_merge(triplet_list):
    relations = set()
    for triplet in triplet_list:
        relations.add(triplet['relation'])
#     print('total relations are ',relations)
    unique_triplet_list = []
    for relation in relations:
        subject = ''
        object_list = []
        for triplet in triplet_list:
            if relation  == triplet['relation']:
                subject = triplet['subject']
                object_list.append(triplet['object'])
        temp_triplet = {}
        temp_triplet['subject'] = subject
        temp_triplet['relation'] = relation
        temp_triplet['object'] = ','.join(object_list)
#         print(temp_triplet)
        unique_triplet_list.append(temp_triplet)
    return unique_triplet_list

In [22]:
def setence_to_triplet(sen):
    with StanfordOpenIE() as client:
#         print('Text: %s.' % sen)
        partial_triplet_list = []
        triplet_list = client.annotate(sen)
        triplet_list = triplet_merge(triplet_list)
        for triple in triplet_list:
            sub = '_'.join(triple['subject'].split())
            rel = relation_camel_casing(triple['relation'].split())
            obj = '_'.join(triple['object'].split())
            partial_triplet = sub + ' | ' + rel + ' | ' + obj
            partial_triplet_list.append(partial_triplet)
        if len(partial_triplet_list) <= 5:
            final_triplet = ' && '.join(partial_triplet_list) + '.'
#         print(final_triplet)
            return final_triplet
        else: 
            return
# sen = "Fred Astaire was born in Omaha, Nebraska, to Johanna (Geilus) and Fritz Austerlitz, a brewer. "
# setence_to_triplet(sen)

Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-0560ac7337304932.props -preload openie


'Fred_Astaire | wasBornTo | Geilus,Johanna && Fred_Astaire | wasBornIn | Omaha,Nebraska && Fred_Astaire | was | born_in_Omaha_to_Johanna,born.'

In [23]:
def text_to_triplet_sentence(text):
    global sentence_list
    global triplet_list
    sents = text.split('.')
    for sent in sents:
        triplet = setence_to_triplet(sent)
        if triplet:
            triplet_list.append(triplet)
            sentence_list.append(sent)

In [24]:
sentence_list = []
triplet_list = []
def dataframe_formatting(df):
    global sentence_list
    global triplet_list
    df['resolved_text'].apply(text_to_triplet_sentence)
#     print(sentence_list,triplet_list)
    final_df = pd.DataFrame(list(zip(triplet_list,sentence_list)),columns=['input_text','target_text'])
    final_df.to_csv('data/final_bio_with_merged_triplets.csv',index = False)
    display(final_df.head())
    print(len(final_df))

In [25]:
# df = pd.read_csv('data/resolved_bio.csv')
df = pd.read_csv('data/wiki_summary_resolved.csv')
display(df.head())
dataframe_formatting(df)

,resolved_text
0,"Fred Astaire was born in Omaha, Nebraska, to J..."
1,One of Hollywood's preeminent male stars of al...
2,Born to Alice Cooper and Charles Cooper. Gary ...
3,"Georges Delerue was born on March 12, 1925 in ..."
4,"Born in London, England, John Gielgud trained ..."


Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c9809135f3e343c2.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-584b17b1ce70488e.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7ad5fb16b92a4009.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/*

Exception ignored in: <function StanfordOpenIE.__del__ at 0x7f05e5bf0700>
Traceback (most recent call last):
  File "/home/anurag/.local/lib/python3.8/site-packages/openie/openie.py", line 102, in __del__
    del os.environ['CORENLP_HOME']
  File "/usr/lib/python3.8/os.py", line 691, in __delitem__
    raise KeyError(key) from None
KeyError: 'CORENLP_HOME'


Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-50ca04f7ed6d4689.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-4c7cf26ae9b44986.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-388c670a3e6c4d5e.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/*

Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-10b2e207078c4589.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-8a6e137774bc4a78.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-8f051cf7382c40d3.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/*

...

Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-850b4ab9772c4e0c.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-fd87ae7118b540e3.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-a598c9e7ff13465d.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/*

Exception ignored in: <function StanfordOpenIE.__del__ at 0x7f05e5bf0700>
Traceback (most recent call last):
  File "/home/anurag/.local/lib/python3.8/site-packages/openie/openie.py", line 102, in __del__
    del os.environ['CORENLP_HOME']
  File "/usr/lib/python3.8/os.py", line 691, in __delitem__
    raise KeyError(key) from None
KeyError: 'CORENLP_HOME'


Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-34a36599c391413f.props -preload openie
Starting server with command: java -Xmx8G -cp /home/anurag/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-7bbbdfed43fe4cda.props -preload openie


,input_text,target_text
0,"Fred_Astaire | wasBornTo | Geilus,Johanna && F...","Fred Astaire was born in Omaha, Nebraska, to J..."
1,Fred_Astaire | enteredShowBusinessAt | age_5 &...,Fred Astaire entered show business at age 5
2,"Adele_Astaire | retired | marry,to_marry_in_19...","After his sister, Adele Astaire retired to ma..."
3,RKO | in | Carioca_1933 && Fred_Astaire | bega...,"In RKO's Carioca (1933), Fred Astaire began F..."
4,Fred_Astaire | wasActiveIn | recording && Fred...,"During these years, Fred Astaire was also act..."


5939


In [ ]:
#references 
#1.https://www.tutorialspoint.com/camelcase-in-python